In [9]:
from selenium import webdriver
import pandas as pd
import time
import os
from bs4 import BeautifulSoup

In [10]:
# Declare the lists for each feature to be scraped
repo_url = []
repo_watches = []
repo_sponsors = []

In [11]:
# Potential Scrapes:
# ../issues
# ../pulls
# ../actions => # of workflow runs
# ../pulse => Active pr and Active issues
# ../network/dependencies
# Sponsered
# Watches
# 
# Ones in Bash Script:
# Commits
# Tags/Releases
# Branches
# Languages
# Contributors

def scrape_page(project_url):
    # Add url to list
    repo_url.append(project_url)
    
    # Set up Web Driver
    driver = webdriver.Chrome()
    driver.get(project_url)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    
    # Parse HTML
    project = project_url[19:]
    num_watches = soup.find(href=f"/{project}/watchers").find("strong").text
    
    creator = project.split('/')[0]
    sponsored = "Yes" if soup.find(href=f"/sponsors/{creator}") else "No"
    
    # Add to list
    repo_watches.append(num_watches)
    repo_sponsors.append(sponsored)
    
    # Close window
    driver.close()

In [12]:
#df = pd.read_excel('projects.xlsx') # can also index sheet by name or fetch all sheets
#project_list = df['Project'].tolist()

#for project in project_list:
    #scrape_page(project)

scrape_page("https://github.com/twbs/bootstrap")
scrape_page("https://github.com/freeCodeCamp/freeCodeCamp")

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /home/dchen117/.cache/selenium/chrome/linux64/122.0.6261.94/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x56074d0dee93 <unknown>
#1 0x56074cdd6ce6 <unknown>
#2 0x56074ce0b6fa <unknown>
#3 0x56074ce076ed <unknown>
#4 0x56074ce5071c <unknown>
#5 0x56074ce43c53 <unknown>
#6 0x56074ce14db3 <unknown>
#7 0x56074ce1577e <unknown>
#8 0x56074d0a47cb <unknown>
#9 0x56074d0a87e5 <unknown>
#10 0x56074d0920e1 <unknown>
#11 0x56074d0a9372 <unknown>
#12 0x56074d0761bf <unknown>
#13 0x56074d0cd488 <unknown>
#14 0x56074d0cd683 <unknown>
#15 0x56074d0de044 <unknown>
#16 0x7f11c03fdac3 <unknown>


In [47]:
projects_df = pd.DataFrame({'Project URL':repo_url, 
                            'Number of Watches':repo_watches,
                            'Sponsored':repo_sponsors})

In [48]:
projects_df

,Project URL,Number of Watches,Sponsored
0,https://github.com/twbs/bootstrap,6.8k,Yes
1,https://github.com/freeCodeCamp/freeCodeCamp,8.5k,Yes


In [ ]:
try:
    with pd.ExcelWriter(
        "project_HTMLfeatures.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("project_HTMLfeatures.xlsx", index=False)